Much of this content can be attributed to the work of Chris Fonnesbeck with source data found at: https://github.com/fonnesbeck/Bios8366

# Model Development & Evaluation

### Alvin D. Jeffery, Lisianne Pruinelli, & Chris Cruz

#### 2018 Nursing Knowledge: Big Data Science Pre-Conference

#### 6/13/18

**Objectives:**  
1. Describe at least 3 modeling/machine learning techniques used in biomedical data science.  
2. Develop a machine learning model for predicting a healthcare outcome.  

## What is Machine Learning (ML)?

Machine Learning (ML) is about coding programs that automatically adjust their performance from exposure to information encoded in data. This learning is achieved via **tunable parameters** that are automatically adjusted according to performance criteria.

Machine Learning can be considered a subfield of Artificial Intelligence (AI).

There are three major classes of ML:

**Supervised learning**: Algorithms which learn from a training set of *labeled* examples (exemplars) to generalize to the set of all possible inputs.  
Examples: regression, support vector machines

**Unsupervised learning**: Algorithms which learn from a training set of *unlableled* examples, using the features of the inputs to categorize inputs together according to some statistical criteria.  
Examples: k-means clustering, kernel density estimation

**Reinforcement learning**: Algorithms that learn via reinforcement from a *critic* that provides information on the quality of a solution, but not on how to improve it. Improved solutions are achieved by iteratively exploring the solution space.

# Introduction to `Scikit-learn`

The `scikit-learn` package is an open-source library that provides a robust set of machine learning algorithms for Python. It is built upon the core Python scientific stack (*i.e.* NumPy, SciPy, Cython), and has a simple, consistent interface, making it useful for many data science applications.

<img src="http://1.bp.blogspot.com/-ME24ePzpzIM/UQLWTwurfXI/AAAAAAAAANw/W3EETIroA80/s1600/drop_shadows_background.png" width="90%"/>

In [ ]:
# previously loaded modules 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib as mplot
%matplotlib inline
import IPython
from IPython.core.display import HTML
from IPython.core.debugger import set_trace
import xlrd

In [ ]:
# load scikit-learn modules
from sklearn import preprocessing
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import random as rnd
from random import random, randint

## Representing Data in `scikit-learn`

Most machine learning algorithms implemented in scikit-learn expect data to be stored in a
**two-dimensional array or matrix**.  The arrays can be
either ``numpy`` arrays, or in some cases ``scipy.sparse`` matrices.
The size of the array is expected to be `[n_samples, n_features]`

- **n_samples:**   The number of samples: each sample is an item to process (e.g. classify).
  A sample can be a document, a picture, a sound, a video, an astronomical object,
  a row in database or CSV file,
  or whatever you can describe with a fixed set of quantitative traits.
- **n_features:**  The number of features or distinct traits that can be used to describe each
  item in a quantitative manner.  Features are generally real-valued, but may be boolean or
  discrete-valued in some cases.

The number of features must be fixed in advance. However it can be very high dimensional
(e.g. millions of features) with most of them being zeros for a given sample. 

In [ ]:
# load data created from previous steps
import os, shutil
cwd = os.getcwd()
datadir = cwd + '/data_oh'

df = pd.read_pickle(datadir + '/data_cleaned_oh.pkl')
df.head()

Most of the models in scikit-learn require the categorical variables be turned into numeric variables.  There are two approaches to this:
1. One Hot Encoding - each item in the categorical variable is turned into its own variable represetnting the presence or abscence of that item.  For example, 'Gender' would turn into 2 variables:  'Gender_M' and 'Gender_F'
2. Label Encoding - assign an integer to each item.  For the 'Gender' variable, 0 might mean Male and 1 might mean Female.

We will use the `LabelEncoder` transformation to change categorical variables into integers.  As a convenience, we can also keep the original (human readable) variable in the Dataframe.

In [ ]:
# Let's use the following variables as our initial set of predictors
cat_cols = ['gender', 'marital', 'race', 'ethnicity']
cat_cols_encoded = [c + '_encoded' for c in cat_cols]
numeric_cols = ['prior_opioid_abuse_diag', 'age', 'opioid_discharge_days_supply']
pred_cols = numeric_cols + cat_cols_encoded
target_col = 'overdose'
all_cols = cat_cols+numeric_cols+[target_col]

df_opioids = df[df['prescribed_opioids'] == 1]

# Encode the categorical variables
dfe = df_opioids[cat_cols]

# Replace missing data with an 'Unknown' category 
# so the missing data will also be encoded
dfe = dfe.replace(np.NaN,'Unknown')

# Encode the categorical variables
encoded = dfe.apply(preprocessing.LabelEncoder().fit_transform)

# Append the non-categorical variables and the encoded variables 
# into a single Dataframe
# Name the new variables as <name>_encoded
dfe = pd.concat([df_opioids[all_cols], encoded.add_suffix('_encoded')],axis=1)
display(dfe.head(2))

In [ ]:
# let's build a model using this set of variables...
pred_cols = ['age', 'opioid_discharge_days_supply', 
             'prior_opioid_abuse_diag', \
             'gender_encoded', 'marital_encoded', 'race_encoded', \
             'ethnicity_encoded']
#pred_cols = ['prior_abuse_diag', 'adult', 'age_at_visit', \
#             'opioid_discharge_days_supply', 'gender_encoded', \
#             'marital_encoded', 'race_encoded', 'ethnicity_encoded']

LR_pred_cols = pred_cols
X = dfe[pred_cols].as_matrix()
y = dfe['overdose'].as_matrix()
print('Using predictor variables of:',pred_cols)

## How do we approach problems from a Data Science perspective?

Imagine a set of observational (empirical data) that we want to *learn* from...  
<img src="https://www.learnopencv.com/wp-content/uploads/2017/02/data-points.png", width='60%'/>

We can fit a variety of models ranging from extremely *simple* to highly *complex* models, e.g., 
<img src='https://www.learnopencv.com/wp-content/uploads/2017/02/bias-variance-tradeoff.png' width='60%'/>

**What are potential problems with each of these cases?**

When applying these same models to a *new* set of data we held out for testing...
<img src='https://www.learnopencv.com/wp-content/uploads/2017/02/bias-variance-tradeoff-test-error.png' width='60%'/>
**We were overfit with the more complex, polynomial model.**

### Bias vs. Variance
<img src='https://www.learnopencv.com/wp-content/uploads/2017/02/Bias-Variance-Tradeoff-In-Machine-Learning-1.png' width='50%'/>

**Rule of Thumb:** Fit model complexity to the data resources (not the target complexity)  

### Practical Approaches  
In Data Science, we tend to do the following with a data set:  
1. Learn a model based on training data (e.g., 60% of data)  
2. Iteratively modify model based on a validation set:  
    2a. Cross-validation/bootstrap with the training data  
    2b. Separate validation set (e.g., 20% of data)  
3. Estimate generalization error with a test set (e.g., 20% of data) that you only look at once  

*Food for Thought:* With small validation sets, error measure is a bad estimator of the best hypothesis.  With large validation sets, error measure is a great estimator of a terrible hypothesis.  

In [ ]:
# Create the training and test datasets
train, test = train_test_split(dfe, test_size=0.25, random_state=123)

X_train = train[pred_cols].as_matrix()
y_train = train['overdose'].as_matrix()
X_test = test[pred_cols].as_matrix()
y_test = test['overdose'].as_matrix()

print('X_train shape = ',X_train.shape)
print('y_train shape = ',y_train.shape)
print('X_test shape = ',X_test.shape)
print('y_test shape = ',y_test.shape)

## Let's Build Some Models!

Let's begin preparing our pain data for machine learning algorithms.

In [ ]:
# models we'll consider
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

### How do these models work?  

#### Logistic Regression
<img src='http://www.saedsayad.com/images/LogReg_1.png' width='80%'/>

#### Linear Discriminant Analysis
<img src='http://sebastianraschka.com/images/blog/2014/linear-discriminant-analysis/lda_1.png' width='80%'/>

#### K-Nearest Neighbors
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/220px-KnnClassification.svg.png' width='30%'/>
The test sample (green circle) should be classified either to the first class of blue squares or to the second class of red triangles. If k = 3 (solid line circle) it is assigned to the second class because there are 2 triangles and only 1 square inside the inner circle. If k = 5 (dashed line circle) it is assigned to the first class (3 squares vs. 2 triangles inside the outer circle).

#### Decision Trees
<img src='https://qph.fs.quoracdn.net/main-qimg-b17755d2e0ffb326d8c39b7f3e07e03b-c' width='80%'/>

#### Random Forest
<img src='https://i.ytimg.com/vi/ajTc5y3OqSQ/hqdefault.jpg' width='65%'/>

#### Gaussian Naive Bayes
<img src='https://chrisalbon.com/images/machine_learning_flashcards/Gaussian_Naive_Bayes_Classifier_print.png' width='80%'/>

## Let's (Really) Build Some Models! 

In [ ]:
# Let's try a simple logistic regression model to see how predictive our data is

# perform a model fit on the training set
LR = LogisticRegression()
result = LR.fit(X_train, y_train)

# calculate predicted values from the model to compare with actual outcomes
expected = y_train
predicted = LR.predict(X_train)

print('\nClassification Report\n',metrics.classification_report(expected, predicted))
print('\nConfusion Matrix\n',metrics.confusion_matrix(expected, predicted))
print('\nAccuracy score =',metrics.accuracy_score(expected, predicted))
print('\nAUC score =',metrics.roc_auc_score(expected, predicted))
print('\nf1 score =',metrics.f1_score(expected, predicted))

### What do these numbers mean?  
According to Wikipedia:

![Confusion Matrix](images/Sensitivity-Wikipedia.png)

Precision = $\frac{tp}{tp + fp}$ ,  
where $tp$ is the number of true positives and $fp$ the number of false positives. The precision is intuitively the ability of the classifier to not label a sample as positive if it is negative.

Recall = $\frac{tp}{tp + fn}$  
The recall is intuitively the ability of the classifier to find all the positive samples.

Accuracy = $\frac{tp + tn}{N}$  

$F1 = 2 * \frac{precision * recall}{precision + recall}$  
The $F1$ score can be interpreted as a weighted average of the precision and recall, where an $F1$ score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the $F1$ score are equal. 

The support is the number of occurrences of each class in $y_{test}$.

In [ ]:
# Lets graph an ROC curve for the model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# we'll use the test set (rather than training) for this evaluation
auc = roc_auc_score(y_test, LR.predict(X_test))
probs = LR.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, probs)
tpr[1] = tpr[0]

plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Model')
plt.legend(loc="lower right")

### Creating a Validation Set from Training Set  

*(Images from James, G., Witten, D., Hastie, T., & Tibshirani, R. (2013). An introduction to statistical learning (Vol. 112). New York: springer.)*

#### Basic Idea

![Basic Split](./images/basic_split.png)

#### Leave One Out Cross Validation (LOOCV) - similar to JackKnife  
![loocv](images/loocv.png)

#### k-Fold Cross Validation 
![kfold](images/cv_5k.png)

#### Bootstrap  
![bootstrap](images/bootstrap.png)

In [ ]:
# prepare configuration for cross validation test harness 
# (from https://machinelearningmastery.com/compare-machine-learning-algorithms-python-scikit-learn/)
seed = 123
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('NB', GaussianNB()))

# evaluate each model in turn
results = []
names = []
scoring = 'roc_auc' 
# others include: 'accuracy', 'f1', 'roc_auc', 
# or found here: http://scikit-learn.org/stable/modules/model_evaluation.html

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, 
                                                 cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
plt.ylabel(scoring)
ax.set_xticklabels(names)
plt.show()

### Which model did "best"?  

We'll tackle this more in the next section on **Model Performance and Evaluation.**  

For now, just let's say *higher* is *better*.  

### *Exercise:* Look at F1 scores instead of AUC scores. 

In [ ]:
# evaluate each model in turn
results = []
names = []

####### EDIT HERE #######
#scoring = 
#########################

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, 
                                                 cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
plt.ylabel(scoring)
ax.set_xticklabels(names)
plt.show()

### Hyperparameter Tuning

In [ ]:
# look at the default settings you used
models

### *Exercise:* Attempt parameter tuning on your own



In [ ]:
# use Logistic Regression & Random Forests 
models = []

####### EDIT HERE #######
#models.append(('LR', LogisticRegression()))
#models.append(('RF', RandomForestClassifier()))
#########################

# evaluate each model in turn
results = []
names = []

####### EDIT HERE #######
#scoring = 
#########################

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, 
                                                 cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

### scikit-learn can help with parameter tuning

In [ ]:
### automated grid search
from sklearn.model_selection import GridSearchCV

param_grid = [
        {'n_estimators': [50, 100, 250], 
         'class_weight': [None, 'balanced'], 
         'max_features': [2, 'sqrt', None]}
]

rfc = RandomForestClassifier()
grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)
cvres = grid_search.cv_results_

In [ ]:
# print results
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(round(np.sqrt(mean_score), 3), params)

## Finalizing the Model

In [ ]:
# assign parameters from best fit
final_fit = RandomForestClassifier(class_weight=None, 
                                   max_features=2, 
                                   n_estimators=100)

final_fit.fit(X_train, y_train)

# store predicted values using the final model
pred_train = final_fit.predict(X_train)
pred_test = final_fit.predict(X_test)

In [ ]:
# explore performance on training data
pd.crosstab(y_train, pred_train, 
            rownames=["Actual"], colnames=["Predicted"])

In [ ]:
# explore performance on testing data
pd.crosstab(y_test, pred_test, 
            rownames=["Actual"], colnames=["Predicted"])

In [ ]:
# Show how to use the resulting model to predict opioid overdose
# age, opioid_discharge_days_supply, prior_opioid_abuse_diag, 
# gender (F), marital (M), race (white), ethnicity (english)
new_patient = [45,10,1,0,1,4,7]

pred = final_fit.predict(np.asmatrix(new_patient))
if pred[0] == 0:
    print('Patient has no overdose risk.')
elif pred[0] == 1:
    print('Patient has overdose risk.')

---
## References

- [`scikit-learn` user's guide](http://scikit-learn.org/stable/user_guide.html)
- Vanderplas, J. (2016) [Python Data Science Handbook: Essential Tools for Working with Data](http://shop.oreilly.com/product/0636920034919.do). O'Reilly Media.